In [ ]:
# for reading in lists of files in folders 
from os import listdir 
from os.path import isfile, join

# for working with numpy functionalities
import numpy as np

# for plotting with matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# for mathematical operations
import math 

# for interpolating points
from scipy.interpolate import interp1d 

# for Procrustes analysis
from scipy.spatial import procrustes #

# for principal component analysis
from sklearn.decomposition import PCA 

# for dataframe capabilities
import pandas as pd

# for hue and other plotting capabilities
import seaborn as sns

# to save csv 
import csv

In [ ]:
##########
# FUNCTION
##########

# define a function to return equally spaced, interpolated points for a given polyline
# inputs: arrays of x and y values for a polyline, number of points to interpolate
# ouputs: interpolated points along the polyline, inclusive of start and end points, as arrays

def interpolation(x, y, number): 

    distance = np.cumsum(np.sqrt( np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2 ))
    distance = distance/distance[-1]

    fx, fy = interp1d( distance, x ), interp1d( distance, y )

    alpha = np.linspace(0, 1, number)
    x_regular, y_regular = fx(alpha), fy(alpha)
    
    return x_regular, y_regular


In [ ]:
# provide path to shapes folder
shapes_path = "file-path"

# retrieve a list of shape file names
shape_files = [f for f in listdir(shapes_path) if isfile(join(shapes_path, f))] 
# sort the file names alphanumerically so they correspond to landmark files
shape_files.sort()

# create a list to store names
shapes = []

# for each shape file
for i in range(len(shape_files)):
    
    # index the current file name, and index the first two characters and append to the shapes list
    shapes.append(shape_files[i][0:-5])
    
# a list of arrays, to store 2D arrays of shape data points for each shape
shape_data = []

# a list of arrays, to store 2D arrays of landmark points for each shape
landmark_data = []

# for each shape
for i in range(len(shapes)):
    
    # load in the shape data as a 2D numpy array
    current_shape_data = np.loadtxt(shapes_path + "/" + shape_files[i])
    
    resolution = np.shape(current_shape_data)[0]
    
    # increase leaf "resolution"
    high_res_x, high_res_y = interpolation(current_shape_data[:,0], current_shape_data[:,1], resolution)
    
    highres_shape = np.column_stack((high_res_x, high_res_y))
    
    pca = PCA(n_components=2)
    PCs = pca.fit_transform(highres_shape)
    
    # append the 2D array to the shape_data list
    shape_data.append(PCs)


    origin = PCs[0,:]

    if origin[0] > 0:
        min_ind = np.argmin(PCs[:,0],axis=None,out=None)
        tip = PCs[min_ind,:]
    else:
        max_ind = np.argmax(PCs[:,0],axis=None,out=None)
        tip = PCs[max_ind,:]
        
    current_landmarks = np.row_stack((origin, tip))
    
    landmark_data.append(current_landmarks)
    
    print(current_shape_data)


In [ ]:
#print(shape_files)

shapefilesdf = pd.DataFrame(shape_files) 

shapefilesdf.to_csv('file-path-shapefilenames.csv') 

### Find the indices of the points in the shape data that correspond to the base and tip landmarks

In [ ]:
##################################################################################################
# we need to find the points in the shape data file that are closest to the base and tip landmarks
# 1. calculate the euclidean distance of each shape data point to the base and tip landmark
# and store the indices of the shape points, so we know which shape data points to use as landmarks
# 2. use indices to plot out base and tip landmarks and make sure we were successful

###########################################################################################
# 1. calculate the euclidean distance of each shape data point to the base and tip landmark
# and store the indices of the shape points, so we know which shape data points to use as landmarks

# a list of lists,
# to store the indices of the points in the shape file closest to the base and tip landmarks
# for each shape
landmark_indices = []

# for each shape
for i in range(len(shapes)):
    
    # select data points in array for the current shape
    current_shape = shape_data[i]
    
    # select landmark points in array for the current shape
    current_landmarks = landmark_data[i]
    
    # create a list to store the distance of each shape point to the base landmark
    base_distances = []
    # create a list to store the distance of each shape point to the tip landmark
    tip_distances = []
    
    # for the number of points in the shape_data file
    # use np.shape, with "0" to count number of rows, or points in the data
    for j in range(np.shape(current_shape)[0]):
        
        # calculate the euclidean distance of each shape point to
        # the base landmark and append to the base distance list
        base_distances.append(np.sqrt(
            (current_landmarks[0,0]-current_shape[j,0])**2 + 
            (current_landmarks[0,1]-current_shape[j,1])**2))
        
        # calculate the euclidean distance of each shape point to
        # the tip landmark and append to the tipdistance list
        tip_distances.append(np.sqrt(
            (current_landmarks[1,0]-current_shape[j,0])**2 + 
            (current_landmarks[1,1]-current_shape[j,1])**2))
    
    # find the indices of the minimum distance of the base and tip to shape points
    # append a list of the base and tip minimum indices to the list landmark indices
    landmark_indices.append([base_distances.index(min(base_distances)),
               tip_distances.index(min(tip_distances))])

In [ ]:
####################################################################################
# 2. use indices to plot out base and tip landmarks and make sure we were successful

# for each shape
for i in range(16):
    
    # get current plot number for plt.subplot, + 1 because indexing starts at 0
    plot_num = i + 1 
    
    # select data points for the current shape
    current_shape = shape_data[i]
    
    # specify subplot
    plt.subplot(4,4,plot_num) 
    
    # plot shape name above each shape (need to modify so no overlap still)
    plt.title(shapes[i], fontsize=8) 
    
    # use fill to plot each shape, index on the columns of the arrays for x and y coords
    plt.fill(current_shape[:,0], current_shape[:,1])
    
    # use the landmark indices to plot the base landmark in orange
    # the ith index in landmark_indices indicates the index of the shape data point corresponding to base
    plt.scatter(current_shape[landmark_indices[i][0],0], current_shape[landmark_indices[i][0],1], c="orange")
    
    # use the landmark indices to plot the tip landmark in magenta
    # the ith index in landmark_indices indicates the index of the shape data point corresponding to tip
    plt.scatter(current_shape[landmark_indices[i][1],0], current_shape[landmark_indices[i][1],1], c="magenta")
    
    # set axes equal in scale and turn off display axes
    plt.gca().set_aspect("equal")
    plt.axis("off")

In [ ]:
for i in range(16):
    
    # get current plot number for plt.subplot, + 1 because indexing starts at 0
    plot_num = i + 1 
    
    # select data points for the current shape
    current_shape = shape_data[i]
    
    # specify subplot
    plt.subplot(4,4,plot_num) 

    # use fill to plot each shape, index on the columns of the arrays for x and y coords
    plt.fill(current_shape[:,0], current_shape[:,1])
    # set axes equal in scale and turn off display axes
    plt.gca().set_aspect("equal")
    plt.axis("off")

### Reindex the coordinate values so that they begin with the base at index = 0

In [ ]:
############################################################################
# The problem is that the base landmark doesn't start at index position 0
# the base is at an arbitrary index value, and the zero index is also
# at an arbitrary position in the shape. We need to reindex to the base being index = 0
# 1. Reindex the shape data
# 2. Reindex the landmark data

############################
# 1. Reindex the shape data

# a list of arrays, with a 2D array of reindexed x and y coordinates for each shape
reindexed_data = []

# for each shape, s
for s in range(len(shapes)):
    
    # get number of coordinate values for the current shape
    num_coords = np.shape(shape_data[s])[0] 
    
    # get the zeroth index to start at (the base)
    zero_index = landmark_indices[s][0] 

    # create an array to store new x and y vals using np.zeros
    reindexed_arr = np.zeros((num_coords,2)) 

    # for the number of points in the shape, i
    for i in range(num_coords):

        # get current index
        curr_ind = i 
        
        # get the new index to use
        new_ind = (i - zero_index)%num_coords 
        
        # store the value at the new index position
        reindexed_arr[new_ind,:] = shape_data[s][curr_ind,:] 
        
    # append the reindexed array into the shape_data list
    reindexed_data.append(reindexed_arr)


In [ ]:
##############################
# 2. Reindex the landmark data

# a list of lists, with lists of indices for each shape
reindexed_indices = []

# for each shape, s
for s in range(len(shapes)):
    
    # get number of coordinate values
    num_coords = np.shape(shape_data[s])[0] 
    
    # get the zeroth, base index
    zero_index = landmark_indices[s][0]
    
    # get the tip index
    tip_index = landmark_indices[s][1]
    
    # get the new base index (it should be 0 in all cases, by definition)
    new_base = (zero_index - zero_index)%num_coords
    
    # get the new tip index
    new_tip = (tip_index - zero_index)%num_coords
    
    # create new indices list
    new_indices = [new_base, new_tip]
        
    # append the reindexed indices into the reindexed_indices list
    reindexed_indices.append(new_indices)
    

### Rotate shapes to the same angle

In [ ]:
####################################################
# We want to rotate all the shapes to the same angle
# Procrustes analysis will take care of rotation
# however, when we reconstruct the leaves, we want them at
# an angle we have chosen, not an arbitrary one
# so we rotate the data before we do Procrustes alignment

#####################################################################
# 1. Using tip and base landmarks, rotate leaves to the desired angle
# 2. Plot out leaves and check that we were successful

###########
# FUNCTIONS
###########

# define a function to find the angle between 3 points anti-clockwise in degrees, p2 being the vertex
# and p1 being the left hand of the angle and p3 a reference point for directionality
# inputs: three angle points, as tuples
# output: angle in degrees

def angle_between(p1, p2, p3):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    deg1 = (360 + math.degrees(math.atan2(x1 - x2, y1 - y2))) % 360
    deg2 = (360 + math.degrees(math.atan2(x3 - x2, y3 - y2))) % 360
    return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)

# define a function to rotate 2D x and y coordinate points around the origin
# inputs: x and y vals (can take pandas dataframe columns) and the degrees (positive, anticlockwise) to rotate
# target_angle is the angle you want to rotate to, in degrees
# outputs: rotated and y vals

def rotate_points(xvals, yvals, degrees, target_angle):
    
    angle_to_move = target_angle - degrees 
    rads = np.deg2rad(angle_to_move)
    
    new_xvals = xvals*np.cos(rads)-yvals*np.sin(rads)
    new_yvals = xvals*np.sin(rads)+yvals*np.cos(rads)
    
    return new_xvals, new_yvals

#####################################################################
# 1. Using tip and base landmarks, rotate leaves to the desired angle

#################
# set the angle to rotate to in degrees
target_angle = 180
#################

# a list of arrays, a list of 2D arrays containing the rotated x and y coordinates for each shape
rotated_data = [] 

# for the number of shapes
for i in range(len(shapes)):

    # the left hand of the angle, the point that determines the angle, the tip in this case
    determining_point = reindexed_data[i][reindexed_indices[i][1],:] 
    # the vertex of the angle, the point the angle rotates around, the base in this case
    origin_point = reindexed_data[i][reindexed_indices[i][0],:] 
    # the reference side of the angle that determines direction, 
    # we use the base but shift the x value positively in our case
    reference_point = (reindexed_data[i][reindexed_indices[i][0],0]+1,reindexed_data[i][reindexed_indices[i][0],1])

    # using the three calculated points above, calculate the angle in degrees
    angle = angle_between(determining_point, origin_point, reference_point) # calculate current angle

    # use the rotate_points function to rotate the points to the desired new angle
    rotated_xvals, rotated_yvals = rotate_points(reindexed_data[i][:,0], reindexed_data[i][:,1], angle, target_angle)

    # put the x and y coordiantes back together again as a 2D numpy array
    rotated_coords = np.column_stack((rotated_xvals, rotated_yvals))
    
    # append the rotated 2D coordinates for the current shape to the list
    rotated_data.append(rotated_coords)

In [ ]:
######################################################
# 2. Plot out leaves and check that we were successful

# for each shape
for i in range(16):
    
    # get current plot number for plt.subplot, + 1 because indexing starts at 0
    plot_num = i + 1 
    
    # select data points for the current shape
    current_shape = rotated_data[i]
    
    # specify subplot
    plt.subplot(4,4,plot_num) 
    
    # plot shape name above each shape (need to modify so no overlap still)
    plt.title(shapes[i], fontsize=8) 
    
    # use fill to plot each shape, index on the columns of the arrays for x and y coords
    plt.fill(current_shape[:,0], current_shape[:,1])
    
    # use the landmark indices to plot the base landmark in orange
    # the ith index in landmark_indices indicates the index of the shape data point corresponding to base
    plt.scatter(current_shape[reindexed_indices[i][0],0], current_shape[reindexed_indices[i][0],1], c="orange")
    
    # use the landmark indices to plot the tip landmark in magenta
    # the ith index in landmark_indices indicates the index of the shape data point corresponding to tip
    plt.scatter(current_shape[reindexed_indices[i][1],0], current_shape[reindexed_indices[i][1],1], c="magenta")
    
    # set axes equal in scale and turn off display axes
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
#plt.savefig('/Users/asiahightower/Desktop/herbarium_paper_figures/individual_leaves/rotated_leaves.png', bbox_inches='tight')

In [ ]:
# for each shape
for i in range(20):
    
    # get current plot number for plt.subplot, + 1 because indexing starts at 0
    plot_num = i + 1 
    
    # select data points for the current shape
    current_shape = rotated_data[i]
    
    # specify subplot
    plt.subplot(4,5,plot_num) 
    plt.fill(current_shape[:,0], current_shape[:,1])
    plt.gca().set_aspect("equal")
    plt.axis("off")
    
#plt.savefig('/Users/asiahightower/Desktop/20_leaf_images.png', bbox_inches='tight')

### Use interpolation to create pseudolandmarks

In [ ]:
########################################################################################
# 1. before we do Procrustes analysis, we need to create the same number of interpolated
# points that correspond with each other between the tip and the base on each side of the leaf
# 2. Check that the interpolation was done correctly

##########
# FUNCTION
##########

# define a function to return equally spaced, interpolated points for a given polyline
# inputs: arrays of x and y values for a polyline, number of points to interpolate
# ouputs: interpolated points along the polyline, inclusive of start and end points, as arrays

def interpolation(x, y, number): 

    distance = np.cumsum(np.sqrt( np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2 ))
    distance = distance/distance[-1]

    fx, fy = interp1d( distance, x ), interp1d( distance, y )

    alpha = np.linspace(0, 1, number)
    x_regular, y_regular = fx(alpha), fy(alpha)
    
    return x_regular, y_regular


########################################################################################
# 1. before we do Procrustes analysis, we need to create the same number of interpolated
# points that correspond with each other between the tip and the base on each side of the leaf

#################
# set the number of points to interpolate on EACH SIDE of the leaf (total point number will be double)
point_number = 100
#################

# a list of arrays, a list of 2D arrays containing the interpolated x and y coordinates for each shape
interpolated_data = [] 

for i in range(len(shapes)):

    # interpolate between the base to the point before the tip for the given point number
    base_to_tip_x, base_to_tip_y = interpolation(rotated_data[i][reindexed_indices[i][0]:(reindexed_indices[i][1])-1,0], 
                                         rotated_data[i][reindexed_indices[i][0]:(reindexed_indices[i][1])-1,1], 
                                         point_number)

    # interpolate between the tip to the last point for the given point number
    tip_to_base_x, tip_to_base_y = interpolation(rotated_data[i][reindexed_indices[i][1]:,0], 
                                         rotated_data[i][reindexed_indices[i][1]:,1], 
                                         point_number)


    # combine each set of points into 2D numpy arrays of x and y coordinates
    base_to_tip_coords = np.column_stack((base_to_tip_x, base_to_tip_y))
    tip_to_base_coords = np.column_stack((tip_to_base_x, tip_to_base_y))

    # combine both sets of points into a single 2D array for the overall shape
    interpolated_coords = np.row_stack((base_to_tip_coords, tip_to_base_coords))
    
    # append the array of interpolated points to the list
    interpolated_data.append(interpolated_coords)
    
    #print(tip_to_base_coords)


Getting landmark data from interpolated_data array

In [ ]:
print(np.shape(interpolated_data))
print(np.shape(shapes))

In [ ]:
my2darray = np.reshape(interpolated_data, (523, 200*2))
np.shape(my2darray)

In [ ]:
df3 = np.column_stack([shapes,my2darray])
np.shape(df3)

In [ ]:
df_out = pd.DataFrame(df3)

In [ ]:
df_out.to_csv('file-path-landmark_dataframe.csv')

In [ ]:
####################################################
# 2. Check that the interpolation was done correctly

# for each shape
for i in range(16):
    
    # get current plot number for plt.subplot, + 1 because indexing starts at 0
    plot_num = i + 1 
    
    # select data points for the current shape
    current_shape = interpolated_data[i]
    
    # specify subplot
    plt.subplot(4,4,plot_num) 
    
    # plot shape name above each shape (need to modify so no overlap still)
    plt.title(shapes[i], fontsize=8) 
    
    # use scatter to check interpolated points
    plt.scatter(current_shape[:,0], current_shape[:,1], s=1)
    
    # set axes equal in scale and turn off display axes
    plt.gca().set_aspect("equal")
    plt.axis("off")

In [ ]:
####################################################
# 2. Check that the interpolation was done correctly
    # select data points for the current shape
current_shape = interpolated_data[9]
    
    # use scatter to check interpolated points
plt.fill(current_shape[:,0], current_shape[:,1])
    
    # set axes equal in scale and turn off display axes
plt.gca().set_aspect("equal")
plt.axis("off")
#plt.savefig('/Users/asiahightower/Desktop/herbarium_paper_figures/individual_leaves/uppermidwest_michigan.png', bbox_inches='tight')

### Calculate mean Generalized Procrustes Analysis (GPA) shape

Procrustes analysis aligns two shapes to each other by minimizing their overall distance. This Procrustes distance can be used as a measure of how similar to shapes are to each other. When aligning many shapes to each other as we are doing here, we need to find an overall mean leaf to align all leaves to. The algorithm to calculate this mean leaf for many leaves is called Generalized Procrustes Analysis (GPA). The GPA algorithm works by starting with an arbitrary reference leaf, usually the first sample. All leaves are aligned to the reference and a mean is calculated. This new mean leaf is then used as a new reference, again which all leaves are aligned to and a new mean is calculated which serves as the new reference. When the Procrustes distance between the new mean and the old mean reach an arbitrarily low Procrustes distance value, we use the mean as the Procrustes mean to align all the leaves to.

In [ ]:
ref_ind = 0 # select a reference index to calculate procrustes distances to
ref_shape = interpolated_data[ref_ind] # select the reference shape
landmark_number = np.shape(interpolated_data[0])[0] # get number of landmarks from first shape (it's the same for all shapes)

mean_diff = 10**(-30) # set a Procrustes distance between means to stop the algorithm, arbitrarily small

old_mean = ref_shape # for the first comparison between means, set old_mean to an arbitrary reference shape

d = 1000000 # set d, the distance, initially arbitraily high

while d > mean_diff: # set boolean criterion for Procrustes distance between mean to stop calculations
    
    arr = np.zeros( ((len(shapes)),landmark_number,2) ) # empty 3D array: # samples, total number of landmarks, 2 coord vals

    for i in range(len(shapes)): # for each leaf shape 

        s1, s2, distance = procrustes(old_mean, interpolated_data[i]) # calculate procrustes adjusted shape to ref for current leaf
        arr[i] = s2 # store procrustes adjusted shape to array

    new_mean = np.mean(arr, axis=(0)) # calculate mean of all shapes adjusted to reference
    
    s1, s2, d = procrustes(old_mean, new_mean) # calculate procrustes distance of new mean to old mean
    
    print("the Procrustes distance between the old and new mean is", d) # print out difference between mean distance
    
    old_mean = new_mean # set the old_mean to the new_mea before beginning another iteration
    
gpa_mean = new_mean # call the resulting mean leaf "gpa_mean"


#### Check the Generalized Procrustes Analysis mean shape

In [ ]:
plt.plot(gpa_mean[:,0], gpa_mean[:,1])
plt.gca().set_aspect("equal")
plt.axis("off")
#plt.savefig('/Users/asiahightower/Desktop/herbarium_paper_figures/individual_leaves/mean_leaf_US.png', bbox_inches='tight')

### Calculate Procrustes-adjusted coordinates for each leaf against the calculated mean

In [ ]:
landmark_number = np.shape(interpolated_data[0])[0] # get number of landmarks from first shape (it's the same for all)

proc_arr = np.zeros( ((len(shapes)), landmark_number, 2) ) # empty 3D array: # samples, total landmarks, 2 coord vals

for i in range(len(shapes)): # for each leaf shape 

    s1, s2, distance = procrustes(gpa_mean, interpolated_data[i]) # calculate procrustes adjusted shape to ref for current leaf
    proc_arr[i] = s2 # store procrustes adjusted shape to array
    

Save Procurestes shape data 

In [ ]:
np.shape(proc_arr)

In [ ]:
proc_arrdf = np.reshape(proc_arr, (523, 200*2))
np.shape(proc_arrdf)

In [ ]:
df4 = np.column_stack([shapes,proc_arrdf])
np.shape(df4)

In [ ]:
df_out_proc = pd.DataFrame(df4)

In [ ]:
df_out_proc.to_csv('file-path-proc_arrdf_phen.csv')

#### Visually check all Procrustes adjusted leaf shapes, plot with the gpa mean leaf

In [ ]:
plot_col = "k" # set plot color
a = 0.01 # set alpha

for i in range(np.shape(proc_arr)[0]):
    
    curr_leaf = proc_arr[i,:,:]
    
    plt.plot(curr_leaf[:,0], curr_leaf[:,1], c=plot_col, alpha=a) # plot current leaf with color and alpha
    plt.gca().set_aspect("equal")
    
plt.plot(gpa_mean[:,0], gpa_mean[:,1]) # plot the mean leaf

plt.gca().set_aspect("equal")
plt.axis("off")
#plt.savefig('/Users/asiahightower/Desktop/herbarium_paper_figures/individual_leaves/mean_leaf_shadow.png', bbox_inches='tight')

### Morphospace and Inverse Principal Component Analysis

#### Find percent variance for each PC

In [ ]:
# get the number of samples from the list of shapes
num_samples = len(interpolated_data) 

# from the first shape, get the total number of coordinates for each shape
total_length = np.shape(interpolated_data[0])[0] 

# use the reshape function to flatten proc_arr to 2D, so that x and y coordinates are input into the PCA
reshaped_arr = proc_arr.reshape(num_samples, total_length*2) 

# set number of PCs to number of samples, so we can see the percent variance for all PCs
test_pca = PCA(n_components=304)

# fit a PCA
test_PCs = test_pca.fit_transform(reshaped_arr) 

# print out explained variance for each PC (a proportion)
#print(test_pca.explained_variance_ratio_) 

#print out cumulative variance explained by PC (a proportion)
#print(test_pca.explained_variance_ratio_.cumsum()) 

#
print('Explained variation per principal component: {}'.format(test_pca.explained_variance_ratio_))



#### Create an inverse PCA space and calculate PC points to reconstruct

In [ ]:
# define number of desired PCs
# for reconstruction, it's easier to just use 2
# but we can use more PC values if desired

pca = PCA(n_components=2)

# apply the sklearn pca function with desired number of components

PCs = pca.fit_transform(reshaped_arr)

# the inverse transform function is applied

inverse_PCA = pca.inverse_transform(PCs)

# specify PC value to reconstruct
# (this is just a test to make sure we can reconstruct 
# theoretical leaves using only PC values)

PC1_val = 0.5
PC2_val = 0

# perform inverse PCA
    
inv_new = pca.inverse_transform(np.array([PC1_val,PC2_val]))

# extract x and y vals, every other using the step indexing

inv_xvals = inv_new[0::2]
inv_yvals = inv_new[1::2]

# plot the theoretical, reconstructed shape
plt.plot(inv_xvals, inv_yvals)
plt.gca().set_aspect("equal")


### Plot background morphospace  

To reduce the amount of code we are using, we will create a function to plot out a background morphospace. We can always come back to change the specifics of the function if needed.

In [ ]:
# scale, determined by checking, to prevent shape overlap
scale = 0.5

# padding on each side of PC to add extra space to see morphospace
PC1_pad = 0.1
PC2_pad = 0.1

# number of intervals to divide each PC axis
num_intervals = 5 

# get PC1 interval values
PC1_intervals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]) ,num_intervals)

# get PC2 interval values
PC2_intervals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), num_intervals)

for i in PC1_intervals:
    
    for j in PC2_intervals:
        
        # perform inverse PCA
    
        inv_new = pca.inverse_transform(np.array([i,j]))

        # extract x and y vals, every other

        inv_PC1_vals = inv_new[0::2]
        inv_PC2_vals = inv_new[1::2]
          
        # scale so the shapes don't overlap, determine the right scale by checking
        
        scaled_PC1_vals = inv_PC1_vals*scale
        scaled_PC2_vals = inv_PC2_vals*scale
        
        # translate to the PCA point position
        
        trans_PC1_vals = scaled_PC1_vals + i
        trans_PC2_vals = scaled_PC2_vals + j
        
        # plot out the results to make sure it is correct

        plt.fill(trans_PC1_vals, trans_PC2_vals, c="gray", alpha=0.5)
        plt.xlim( (np.min(PCs[:,0])-PC1_pad, np.max(PCs[:,0])+PC1_pad) )
        plt.ylim( (np.min(PCs[:,1])-PC2_pad, np.max(PCs[:,1])+PC2_pad) )
        plt.gca().set_aspect("equal")
#plt.savefig('/Users/asiahightower/Desktop/herbarium_paper_figures/individual_leaves/morphospace_all_5.png', bbox_inches='tight')

In [ ]:
# define a function to plot background morphospace
# inputs: scale, determined by checking, to prevent shape overlap in the plot. 
# PC1_pad and PC2_pad, padding on each side of PC to add extra space to see morphospace
# num_intervals, number of intervals to divide each PC axis as a grid
# output: a morphospace plot

def Morphospace(scale, PC1_pad, PC2_pad, num_intervals, PCs):

    # get PC1 interval values
    PC1_intervals = np.linspace( np.min(PCs[:,0]), np.max(PCs[:,0]) ,num_intervals)

    # get PC2 interval values
    PC2_intervals = np.linspace( np.min(PCs[:,1]), np.max(PCs[:,1]), num_intervals)

    for i in PC1_intervals:

        for j in PC2_intervals:

            # perform inverse PCA

            inv_new = pca.inverse_transform(np.array([i,j]))

            # extract x and y vals, every other

            inv_PC1_vals = inv_new[0::2]
            inv_PC2_vals = inv_new[1::2]

            # scale so the shapes don't overlap, determine the right scale by checking

            scaled_PC1_vals = inv_PC1_vals*scale
            scaled_PC2_vals = inv_PC2_vals*scale

            # translate to the PCA point position

            trans_PC1_vals = scaled_PC1_vals + i
            trans_PC2_vals = scaled_PC2_vals + j

            # plot out the results to make sure it is correct

            plt.fill(trans_PC1_vals, trans_PC2_vals, c="gray", alpha=0.3)
            plt.xlim( (np.min(PCs[:,0])-PC1_pad, np.max(PCs[:,0])+PC1_pad) )
            plt.ylim( (np.min(PCs[:,1])-PC2_pad, np.max(PCs[:,1])+PC2_pad) )
            plt.gca().set_aspect("equal")
        

#### Check to see if the function works

In [ ]:
# scale, determined by checking, to prevent shape overlap
scale = 0.5

# padding on each side of PC to add extra space to see morphospace
PC1_pad = 0.1
PC2_pad = 0.1

# number of intervals to divide each PC axis
num_intervals = 7 

Morphospace(scale, PC1_pad, PC2_pad, num_intervals, PCs)

### Calculate shape descriptors

To measure lobing we will use circularity. We are currently using the circularity less used, which puts the area in the numerator and the perimeter in the denominator, so that 1 is a perfect circle and 0 deviates from that, but we can always switch these. Aspect ratio is calculated as width/length, and we use the fact that our leaves are oriented upright to simply take the min and max width and length values of leaves. We can figure out how to calculate solidity if needed as well, but maybe circularity is enough.

In [ ]:
# Create a function to calculate circularity

# input: a 2D array of points for a polygon
# output: circularity value, calculated as (4*pi*area)/perimeter**2
# where 1 is a perfect circle and down to 0 for non-circular shapes


def Circularity(shape_arr):
    
    lines = np.hstack([shape_arr,np.roll(shape_arr,-1,axis=0)])
    area = 0.5*abs(sum(x1*y2-x2*y1 for x1,y1,x2,y2 in lines))
    
    distance = np.cumsum(np.sqrt( np.ediff1d(shape_arr[:,0], to_begin=0)**2 + np.ediff1d(shape_arr[:,1], to_begin=0)**2 ))
    perimeter = distance[-1]
    
    circularity = (4*math.pi*area)/perimeter**2
    
    return circularity
    
 

#### Calculate circularity and aspect ratio values for all leaves

In [ ]:
# a list to store circularity values
circularity_vals = []

# a list to store aspect ratio values (width divided by length)
aspect_ratio_vals = []

#length values 
length_vals = []

#width values
width_vals = []

# for each Procrustes-adjusted shape
for i in range(np.shape(proc_arr)[0]):
    
    # select current shape
    curr_shape = proc_arr[i] 
    
    # calculate circularity and append to list
    circularity_vals.append(Circularity(curr_shape))
    
    # calculate length, width, and aspect ratio
    length_vals.append(length)
    width_vals.append(width)
    aspect_ratio_vals.append(width/length)

In [ ]:
circdf = pd.DataFrame(circularity_vals, aspect_ratio_vals)

#### Create a pandas dataframe to use seaborn, using a dictionary with the keys as column names and the values the data

In [ ]:
df = pd.DataFrame({"PC1":PCs[:,0], "PC2":PCs[:,1], "circ":circularity_vals, "ar":aspect_ratio_vals})

Save df of pc's to use and combine weather data + plot 

In [ ]:
df.to_csv('file-path-pc_circ_data.csv')

#### Create a PC morphospace plot, points colored by circularity

In [ ]:
# scatterplot of PC values, colored by circularity (using hue in seaborn)
sns.scatterplot(data=df, x="PC1", y="PC2", hue="circ", palette = sns.color_palette("viridis", as_cmap = True))

# scale, determined by checking, to prevent shape overlap
scale = 0.5

# padding on each side of PC to add extra space to see morphospace
PC1_pad = 0.1
PC2_pad = 0.1

# number of intervals to divide each PC axis
num_intervals = 5 

# use the morphospace function to create morphospace
Morphospace(scale, PC1_pad, PC2_pad, num_intervals, PCs)

# place the legend outside of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.savefig('/Users/asiahightower/Desktop/pca_circ_morphospace.png', bbox_inches='tight')

#### Create a PC morphospace plot, points colored by aspect ratio

In [ ]:
# scatterplot of PC values, colored by aspect ratio (using hue in seaborn)
sns.scatterplot(data=df, x="PC1", y="PC2", hue="ar", palette = sns.color_palette("cubehelix", as_cmap = True))

# scale, determined by checking, to prevent shape overlap
scale = 0.4

# padding on each side of PC to add extra space to see morphospace
PC1_pad = 0.1
PC2_pad = 0.1

# number of intervals to divide each PC axis
num_intervals = 5 

# use the morphospace function to create morphospace
Morphospace(scale, PC1_pad, PC2_pad, num_intervals, PCs)

# place the legend outside of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.savefig('/Users/asiahightower/Desktop/pca_aspectratio_morphospace.png', bbox_inches='tight')

### Find closest matches to archetypal leaves

For a given leaf, find its closest match to a set of archetypepal leaves using Procrustes distance

In [ ]:
# Ianetta 7 leaf shapes
# a list of the shapes to compare to
# in this case, I will use A1, B1, C1, and D1
# which are indices 0, 4, 8, and 12

archetypes = [proc_arr[507], proc_arr[508], proc_arr[509], proc_arr[510], proc_arr[511], proc_arr[512], proc_arr[513],
              proc_arr[514], proc_arr[515], proc_arr[516], proc_arr[517], proc_arr[518]]

# create a list of names to refer to them as
names = ["Type1a1", "Type1a2", "Type1a3", "Type1a4", "Type2a1", "Type2a2", "Type3", "Type4", "Type5", "Type6", 
        "Type7a", "Type7b"]

# create a list to store the closest matched archetype to each leaf
closest_matches_iannetta = []

# for each shape
for i in range(len(shapes)):
    
    # select the current shape
    curr_shape = proc_arr[i]
    
    # create a list of distances of the current shape to each archetype
    archetype_dist = []
    
    # for each archetype
    for j in range(len(names)):
        
        # calculate Procrustes distance between archetype and current shape
        s1, s2, distance = procrustes(archetypes[j], curr_shape) 
        
        # append distance to list of distances
        archetype_dist.append(distance)
    
    # find index of closest archetype to current leaf
    archetype_index = archetype_dist.index(min(archetype_dist))
    
    # get name of closest archetype to current leaf using the index
    archetype_name = names[archetype_index]
    
    # append the closest match to list
    closest_matches_iannetta.append(archetype_name)


In [ ]:
# to the dataframe that already has PC1, PC2, circularity, and aspect ratio, and best_match

df["best_match_iannetta"] = closest_matches_iannetta

df.head()

#df.to_csv('/Users/asiahightower/Desktop/bestmatches.csv') 

In [ ]:
# Plot the data and color by best match

# scatterplot of PC values, colored by best_match (using hue in seaborn)
sns.scatterplot(data=df, x="PC1", y="PC2", hue="best_match_iannetta", palette = sns.color_palette('colorblind'))

# scale, determined by checking, to prevent shape overlap
scale = 0.4

# padding on each side of PC to add extra space to see morphospace
PC1_pad = 0.1
PC2_pad = 0.1

# number of intervals to divide each PC axis
num_intervals = 5 

# use the morphospace function to create morphospace
Morphospace(scale, PC1_pad, PC2_pad, num_intervals, PCs)

# place the legend outside of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

#plt.savefig('/Users/asiahightower/Desktop/colorandbestmatch.png', bbox_inches='tight')

In [ ]:
# Shull 4 leaf shapes 
# a list of the shapes to compare to
# in this case, I will use A1, B1, C1, and D1
# which are indices 0, 4, 8, and 12

archetypes = [proc_arr[521], proc_arr[520], proc_arr[522], proc_arr[519]]

# create a list of names to refer to them as
names = ["simplex", "rhomboidea", "tenuis", "heteris"]

# create a list to store the closest matched archetype to each leaf
closest_matches_shull = []

# for each shape
for i in range(len(shapes)):
    
    # select the current shape
    curr_shape = proc_arr[i]
    
    # create a list of distances of the current shape to each archetype
    archetype_dist = []
    
    # for each archetype
    for j in range(len(names)):
        
        # calculate Procrustes distance between archetype and current shape
        s1, s2, distance = procrustes(archetypes[j], curr_shape) 
        
        # append distance to list of distances
        archetype_dist.append(distance)
    
    # find index of closest archetype to current leaf
    archetype_index = archetype_dist.index(min(archetype_dist))
    
    # get name of closest archetype to current leaf using the index
    archetype_name = names[archetype_index]
    
    # append the closest match to list
    closest_matches_shull.append(archetype_name)



In [ ]:
# to the dataframe that already has PC1, PC2, circularity, and aspect ratio, and best_match

df["best_match_shull"] = closest_matches_shull

df.head()

#df.to_csv('/Users/asiahightower/Desktop/bestmatches2.csv') 

In [ ]:
# Plot the data and color by best match

# scatterplot of PC values, colored by best_match (using hue in seaborn)
sns.scatterplot(data=df, x="PC1", y="PC2", hue="best_match_shull", palette = sns.color_palette('colorblind'))

# scale, determined by checking, to prevent shape overlap
scale = 0.4

# padding on each side of PC to add extra space to see morphospace
PC1_pad = 0.1
PC2_pad = 0.1

# number of intervals to divide each PC axis
num_intervals = 5

# use the morphospace function to create morphospace
Morphospace(scale, PC1_pad, PC2_pad, num_intervals, PCs)

# place the legend outside of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#plt.savefig('/Users/asiahightower/Desktop/bestmatchshull.png', bbox_inches='tight')

### plotting by pheno + pop

In [ ]:
pc_metadata = pd.read_csv('/Users/asiahightower/Desktop/desktop_csv_files/herb_pc_circ_data_complete.csv')

In [ ]:
for col in pc_metadata.columns:
    print(col)

In [ ]:
pc_metadata.head()

In [ ]:
df3 = pc_metadata.loc[pc_metadata["type_shull"] == "rhomboidea"]
df3.head()